# Objective

To introduce a *production-aware* modeling workflow to predict the price of diamonds using a set of input features

**Preliminaries**

In [1]:
import sklearn
import joblib
import session_info

import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.tree import DecisionTreeRegressor

In [2]:
session_info.show()

In [3]:
sklearn.set_config(display='diagram')

# Business Background

A popular diamond jeweller with 130 showrooms across the globe is facing a price prediction problem. A common customer question that echoes in their retail outlets is the impact on price because of changes in some aspects of the ornament. For example, usually customers ask: "If I decreased the carat of the diamonds used in this design, by how much would the price reduce?". Such queries often require an expert intervention on the shopfloor and result in a subdued customer experience. 

To solve this problem, you are tasked to *build* and *deploy* a *reliable* model that will be used by sales people in the shopfloor. This model should be able to receive thousands of requests from sales persons in 130 showrooms per day and return a price recommendation within 60 seconds. 

# Modeling

<div class="alert alert-block alert-success">

<b>Task 1 (5 minutes)</b> 
    
Translate the above business problem into a data problem. Specifically, brainstorm and identify:

- the target
- the features
- sources of data
    
</div>

## Data Collection

In a business context, assembling the features and target is a non-trivial task. This step involves extensive discussions with other stakeholders (e.g., product teams, operations teams). A *data engineer* usually handles the orchestration of workflows that are usually referred to as *Extract-Transform-Load (ETL) jobs*. The responsibility to ensure that the data required by the data science team is appropriately assembled rests with the data engineer. From the perspective of a data scientist, it is important however to understand how a training data set is assembled.

In [4]:
dataset = fetch_openml(data_id=43355, as_frame=True, parser='auto')

In [5]:
print(dataset.DESCR)

Context
Buying a diamond can be frustrating and expensive.  
It inspired me to create this dataset of 119K natural and lab-created diamonds from brilliantearth.com to demystify the value of the 4 Cs  cut, color, clarity, carat.
This data was scraped using DiamondScraper.
Content



Attribute
Description
Data Type




id
Diamond identification number provided by Brilliant Earth
int


url
URL for the diamond details page
string


shape
External geometric appearance of a diamond
string/categorical


price
Price in U.S. dollars
int


carat
Unit of measurement used to describe the weight of a diamond
float


cut
Facets, symmetry, and reflective qualities of a diamond
string/categorical


color
Natural color or lack of color visible within a diamond, based on the GIA grade scale
string/categorical


clarity
Visibility of natural microscopic inclusions and imperfections within a diamond
string/categorical


report
Diamond certificate or grading report provided by an independent gemology lab
s

In [6]:
diamond_prices = dataset.data

In [7]:
diamond_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119307 entries, 0 to 119306
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            119307 non-null  int64  
 1   url           119307 non-null  object 
 2   shape         119307 non-null  object 
 3   price         119307 non-null  int64  
 4   carat         119307 non-null  float64
 5   cut           119307 non-null  object 
 6   color         119307 non-null  object 
 7   clarity       119307 non-null  object 
 8   report        119307 non-null  object 
 9   type          119307 non-null  object 
 10  date_fetched  119307 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 10.0+ MB


<div class="alert alert-block alert-success">

<b>Task 2 (5 minutes)</b> 
    
Read through the data description presented in the previous cells carefully. 

From all the dataset features, a subset of features are identified to build the model and presented in the code cells that follow.

Are these appropriate? Are there any important features in the dataset that are excluded?
</div>

In [8]:
target = ['price']
numeric_features = ['carat']
categorical_features = ['shape', 'cut', 'color', 'clarity', 'report', 'type']

## Exploratory Data Analysis

<div class="alert alert-block alert-success">

<b>Task 3 (5 minutes)</b> 
    
Some suggestions for exploratory data analysis are presented below. Are these complete? What other explorations would you suggest?
    
You should present at least 2 most important EDA steps.
    
</div>

In [9]:
diamond_prices.head()

,id,url,shape,price,carat,cut,color,clarity,report,type,date_fetched
0,10086429,https://www.brilliantearth.com//loose-diamonds...,Round,400,0.30,'Very Good',J,SI2,GIA,natural,'2020-11-29 12-26 PM'
1,10016334,https://www.brilliantearth.com//loose-diamonds...,Emerald,400,0.31,Ideal,I,SI1,GIA,natural,'2020-11-29 12-26 PM'
2,9947216,https://www.brilliantearth.com//loose-diamonds...,Emerald,400,0.30,Ideal,I,VS2,GIA,natural,'2020-11-29 12-26 PM'
3,10083437,https://www.brilliantearth.com//loose-diamonds...,Round,400,0.30,Ideal,I,SI2,GIA,natural,'2020-11-29 12-26 PM'
4,9946136,https://www.brilliantearth.com//loose-diamonds...,Emerald,400,0.30,Ideal,I,SI1,GIA,natural,'2020-11-29 12-26 PM'


In [10]:
diamond_prices.loc[:, target].describe()

,price
count,1.193070e+05
mean,3.286843e+03
std,9.114695e+03
min,2.700000e+02
25%,9.000000e+02
50%,1.770000e+03
75%,3.490000e+03
max,1.348720e+06


In [11]:
diamond_prices.loc[:, numeric_features].describe()

,carat
count,119307.000000
mean,0.884169
std,0.671141
min,0.250000
25%,0.400000
50%,0.700000
75%,1.100000
max,15.320000


In [12]:
diamond_prices.loc[:, categorical_features].describe()

,shape,cut,color,clarity,report,type
count,119307,119307,119307,119307,119307,119307
unique,10,5,7,8,4,2
top,Round,'Super Ideal',E,VS1,GIA,natural
freq,76080,55244,24730,27259,68782,70313


Write your code below. Add more cells as required.

## Model Building

In [13]:
X = diamond_prices.drop(columns=target)
y = diamond_prices[target]

In [14]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
                                                test_size=0.2,
                                                random_state=20130810)

In [15]:
Xtrain.to_csv('data/20230104_training_features.csv', index=False)
ytrain.to_csv('data/20230104_training_target.csv', index=False)

Versioning data is an equally important component of building models with deployment as an end goal.

### Model v1

<div class="alert alert-block alert-success">

<b>Task 4 (10 minutes)</b> 
    
Using the training data above, estimate a [`DecisionTree`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) for the target (`price`) using the numeric feature (`caret`) mentioned above.

Extra credit if you can finish hyperparameter tuning as well!
</div>

Write your code below (add more cells as required).

### Suggested model building workflow

Remember that the model we build is eventually going to be deployed for production. In this context, it is important to include all the preprocessing that you expect to do with the model itself. `scikit-learn` has several production-ready capabilities to help with this workflow. A common production-aware training workflow is to use a [model building pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). Let us see how a model pipeline can be set up.

In [16]:
numeric_features

['carat']

The [`make_column_transformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_transformer.html) is a great way to specify the preprocessing steps (e.g., scaling, missing value imputation) to be conducted for specific features. This transformer automates the `fit` and `transform` methods during the training and prediction steps.

In [17]:
preprocessor = make_column_transformer((StandardScaler(), numeric_features))

In [18]:
model_pipeline = make_pipeline(preprocessor, DecisionTreeRegressor())

Once built, a model pipeline can be visualized like so:

In [19]:
model_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['carat'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

In this case, we do not conduct any specific hyperparameter tuning. More generally, we would use the model pipeline to estimate the best hyperparameter for the model using cross-validation.

In [20]:
model_pipeline.fit(Xtrain, ytrain)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['carat'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

For each model we estimate, we save the entire pipeline, that is the model configuration along with all the steps conducted in preprocessing.

In [21]:
joblib.dump(model_pipeline, 'models/model-v1.joblib')

['models/model-v1.joblib']

Saving an estimated model allows us to reuse the model for post-estimation checks. Crucial components of a model are often referred to as *artefacts*. In this case, the model artefact is the estimated model pipeline.

The model is now saved in a binary object (`.joblib`) and the process of converting the model (Python object) to a binary format is referred to as *serialization*.

### Testing predictions 

When testing an estimated model, it is important to disassociate ourselves from the model building process. **We should not use training data during this phase.**

In [22]:
saved_model = joblib.load('models/model-v1.joblib')

In [23]:
saved_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['carat'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

Note how the saved model includes the preprocessing pipeline. Let us now test this model with sample data.

<div class="alert alert-block alert-success">

<b>Task 5 (5 minutes)</b> 
    
The following code allows us to test the model out on a selected sample using the saved model. 

Observe the descriptive statistics for the feature (carat) and the target (price) below.

</div>

In [24]:
diamond_prices.carat.describe()

count    119307.000000
mean          0.884169
std           0.671141
min           0.250000
25%           0.400000
50%           0.700000
75%           1.100000
max          15.320000
Name: carat, dtype: float64

In [25]:
diamond_prices.price.describe()

count    1.193070e+05
mean     3.286843e+03
std      9.114695e+03
min      2.700000e+02
25%      9.000000e+02
50%      1.770000e+03
75%      3.490000e+03
max      1.348720e+06
Name: price, dtype: float64

<div class="alert alert-block alert-success">

<b>Test case 1</b>

One test case (test case 1) is presented below. Why is test case 1 badly formulated and how would you fix it?
    
</div>

*When writing test cases, think in terms of model input and output. Forget about the specifics of the model architecture.* 

In [26]:
sample = {'carat': 0.02}

In [27]:
pd.DataFrame([sample])

,carat
0,0.02


In [28]:
saved_model.predict(pd.DataFrame([sample]))

array([634.87256372])

<div class="alert alert-block alert-success">

<b>Test cases 2 and 3</b>
- Think of 2 good test cases for model v1 and fill in the space provided with these test cases
- Is the model performance in line with what you would expect from a business context?    
</div>

**Test case 2**

**Test case 3**

## Model v2

Let us now look at how categorical features are handled with model pipelines.

### Suggested model building workflow

In [29]:
numeric_features

['carat']

Categorical features should be handled appropriately using a [one-hot encoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html). The default behavior is to exclude features which are not explicitly dealt with the preprocessor. This allows us to keep the training data separate from the preprocessing pipeline.

Remember that in a production setting, the model might be exposed to categories it has not seen in the training phase. We will need a robust method to return meaningful predictions even in this case.

In [30]:
categorical_features_model2 = ['type']

In [31]:
preprocessor = make_column_transformer((StandardScaler(), numeric_features),
                                       (OneHotEncoder(handle_unknown='ignore'),
                                        categorical_features_model2))

The preprocessor now has two steps depending on the type of the column. If the feature is numeric it passes through the scaler and if it is categorical it passes through the one-hot encoder. The argument `handle_unknown='ignore'` ensures that if new levels are encountered, they are encoded as 0's.

In [32]:
model_pipeline = make_pipeline(preprocessor, DecisionTreeRegressor())

In [33]:
model_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(), ['carat']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['type'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

In [34]:
model_pipeline.fit(Xtrain, ytrain)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(), ['carat']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['type'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

In [35]:
joblib.dump(model_pipeline, 'models/model-v2.joblib')

['models/model-v2.joblib']

### Testing predictions 

In [36]:
saved_model = joblib.load('models/model-v2.joblib')

In [37]:
saved_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(), ['carat']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['type'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

<div class="alert alert-block alert-success">

<b>Task 6 (5 minutes)</b> 
    
The following code allows us to test the model out on a selected sample using the saved model. Address the issue with test case 1 and write out a test based on the specification for test 2 below.

</div>

In [38]:
diamond_prices.carat.describe()

count    119307.000000
mean          0.884169
std           0.671141
min           0.250000
25%           0.400000
50%           0.700000
75%           1.100000
max          15.320000
Name: carat, dtype: float64

In [39]:
diamond_prices.type.value_counts()

natural    70313
lab        48994
Name: type, dtype: int64

In [40]:
diamond_prices.price.describe()

count    1.193070e+05
mean     3.286843e+03
std      9.114695e+03
min      2.700000e+02
25%      9.000000e+02
50%      1.770000e+03
75%      3.490000e+03
max      1.348720e+06
Name: price, dtype: float64

<div class="alert alert-block alert-success">

<b>Test case 1</b> 
    
One test case is presented below. Is this a bad test case? (Hint: Look at the value counts)
    
</div>

In [41]:
diamond_prices.type.value_counts()

natural    70313
lab        48994
Name: type, dtype: int64

In [42]:
sample = {'carat': 0.02, 'type': 'Z'}

In [43]:
pd.DataFrame([sample])

,carat,type
0,0.02,Z


In [44]:
saved_model.predict(pd.DataFrame([sample]))

array([634.87256372])

We can inspect the transformation executed by the one-hot encoder as a part of the pipeline like so:

In [45]:
saved_model[:-1].transform(pd.DataFrame([sample]))

array([[-1.28466235,  0.        ,  0.        ]])

<div class="alert alert-block alert-success">

<b>Test case 2</b> 
    
Implement the following test case for model v2: Natural diamonds with the same carat should have more price than those synthesized in the lab.

Is the model performance in line with what you would expect from a business context?
    
</div>


## Model v3

Let us now use all the features to estimate the model.

### Suggested model building workflow

Like we have done for model v1 and model v2, we use standard scaling for numeric features and one hot encoding for categorical features.

In [46]:
preprocessor = make_column_transformer((StandardScaler(), numeric_features),
                                       (OneHotEncoder(handle_unknown='ignore'), 
                                                      categorical_features))

In [47]:
model_pipeline = make_pipeline(preprocessor, DecisionTreeRegressor())

In [48]:
model_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(), ['carat']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['shape', 'cut', 'color',
                                                   'clarity', 'report',
                                                   'type'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

In [49]:
model_pipeline.fit(Xtrain, ytrain)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(), ['carat']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['shape', 'cut', 'color',
                                                   'clarity', 'report',
                                                   'type'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

In [50]:
joblib.dump(model_pipeline, 'models/model-v3.joblib')

['models/model-v3.joblib']

### Testing predictions 

In [51]:
saved_model = joblib.load('models/model-v3.joblib')

<div class="alert alert-block alert-success">

<b>Task 7 (5 minutes)</b> 
    
The following code allows us to test the model out on a selected sample using the saved model. 

One test case is presented below. Is it a bad test case?
    
</div>


**Test case 1**

In [52]:
sample = {
    "shape": "oval",
    "carat": 1,
    "cut": "Very Good",
    "color": "J",
    "clarity": "SI2",
    "type": "lab",
    "report": "GIA"
}

In [53]:
pd.DataFrame([sample])

,shape,carat,cut,color,clarity,type,report
0,oval,1,Very Good,J,SI2,lab,GIA


In [54]:
saved_model.predict(pd.DataFrame([sample]))

array([990.])

In [55]:
saved_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(), ['carat']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['shape', 'cut', 'color',
                                                   'clarity', 'report',
                                                   'type'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

## Final Model

In [56]:
saved_model_files = ['model-v1.joblib', 'model-v2.joblib', 'model-v3.joblib']

In [57]:
print("Mean Absolute Error (MAE) on test set:")
for saved_model_file in saved_model_files:
    saved_model = joblib.load(saved_model_file)
    print(saved_model_file)
    print(sklearn.metrics.median_absolute_error(ytest, 
                                                saved_model.predict(Xtest)))

Mean Absolute Error (MAE) on test set:
model-v1.joblib
595.462457589615
model-v2.joblib
288.9296081277213
model-v3.joblib
76.66666666666652


<div class="alert alert-block alert-success">

<b>Task 8 (5 minutes)</b> 
    
- Why did we use MAE for model evaluation instead of RMSE?
- Which model would you recommend for production? 
    
</div>


In [58]:
Xtest.to_csv('data/20230104_test_features.csv', index=False)
ytest.to_csv('data/20230104_test_target.csv', index=False)